# Linear regression

In [1]:
%load_ext sql

In [2]:
# Greenplum Database 5.x on GCP (PM demo machine) - via tunnel
%sql postgresql://gpadmin@localhost:8000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

In [3]:
%sql select madlib.version();

1 rows affected.


version
"MADlib version: 1.18.0-dev, git revision: rel/v1.17.0-100-g4987e8f, cmake configuration time: Wed Mar 24 23:51:47 UTC 2021, build type: release, build system: Linux-3.10.0-1160.21.1.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


# 1. Load test data

In [4]:
%%sql 
DROP TABLE IF EXISTS houses;

CREATE TABLE houses (id INT, tax INT, bedroom INT, bath FLOAT, price INT,
            size INT, lot INT);

INSERT INTO houses VALUES   
  (1 ,  590 ,       2 ,    1 ,  50000 ,  770 , 22100),
  (2 , 1050 ,       3 ,    2 ,  85000 , 1410 , 12000),
  (3 ,   20 ,       3 ,    1 ,  22500 , 1060 ,  3500),
  (4 ,  870 ,       2 ,    2 ,  90000 , 1300 , 17500),
  (5 , 1320 ,       3 ,    2 , 133000 , 1500 , 30000),
  (6 , 1350 ,       2 ,    1 ,  90500 ,  820 , 25700),
  (7 , 2790 ,       3 ,  2.5 , 260000 , 2130 , 25000),
  (8 ,  680 ,       2 ,    1 , 142500 , 1170 , 22000),
  (9 , 1840 ,       3 ,    2 , 160000 , 1500 , 19000),
 (10 , 3680 ,       4 ,    2 , 240000 , 2790 , 20000),
 (11 , 1660 ,       3 ,    1 ,  87000 , 1030 , 17500),
 (12 , 1620 ,       3 ,    2 , 118600 , 1250 , 20000),
 (13 , 3100 ,       3 ,    2 , 140000 , 1760 , 38000),
 (14 , 2070 ,       2 ,    3 , 148000 , 1550 , 14000),
 (15 ,  650 ,       3 ,  1.5 ,  65000 , 1450 , 12000);
    
SELECT * FROM houses ORDER BY id;

Done.
Done.
15 rows affected.
15 rows affected.


id,tax,bedroom,bath,price,size,lot
1,590,2,1.0,50000,770,22100
2,1050,3,2.0,85000,1410,12000
3,20,3,1.0,22500,1060,3500
4,870,2,2.0,90000,1300,17500
5,1320,3,2.0,133000,1500,30000
6,1350,2,1.0,90500,820,25700
7,2790,3,2.5,260000,2130,25000
8,680,2,1.0,142500,1170,22000
9,1840,3,2.0,160000,1500,19000
10,3680,4,2.0,240000,2790,20000


# 2. Train regression model
First, we generate a single regression for all data.

In [5]:
%%sql
DROP TABLE IF EXISTS houses_linregr, houses_linregr_summary;

SELECT madlib.linregr_train( 'houses',
                             'houses_linregr',
                             'price',
                             'ARRAY[1, tax, bath, size]'
                           );

SELECT * FROM houses_linregr_summary;

Done.
1 rows affected.
1 rows affected.


method,source_table,out_table,dependent_varname,independent_varname,num_rows_processed,num_missing_rows_skipped,grouping_col
linregr,houses,houses_linregr,price,"ARRAY[1, tax, bath, size]",15,0,None


# 3. Train regression model with grouping
Next we generate three output models, one for each value of "bedroom".

In [6]:
%%sql 
DROP TABLE IF EXISTS houses_linregr_bedroom, houses_linregr_bedroom_summary;

SELECT madlib.linregr_train( 'houses',
                             'houses_linregr_bedroom',
                             'price',
                             'ARRAY[1, tax, bath, size]',
                             'bedroom'
                           );

SELECT * FROM houses_linregr_bedroom_summary;

Done.
1 rows affected.
1 rows affected.


method,source_table,out_table,dependent_varname,independent_varname,num_rows_processed,num_missing_rows_skipped,grouping_col
linregr,houses,houses_linregr_bedroom,price,"ARRAY[1, tax, bath, size]",15,0,bedroom


# 4. Review model

In [7]:
%%sql
SELECT * FROM houses_linregr;

1 rows affected.


coef,r2,std_err,t_stats,p_values,condition_no,num_rows_processed,num_missing_rows_skipped,variance_covariance
"[-12849.4168959872, 28.9613922651775, 10181.6290712649, 50.516894915353]",0.768577580597,"[33453.0344331377, 15.8992104963991, 19437.7710925915, 32.9280231740856]","[-0.384103179688204, 1.82156166004197, 0.523806408809163, 1.53416118083608]","[0.708223134615411, 0.0958005827189556, 0.610804093526516, 0.153235085548177]",9002.5045707,15,0,"[[1119105512.7847, 217782.067878005, -283344228.394538, -616679.693190829], [217782.067878005, 252.784894408806, -46373.1796964038, -369.864520095145], [-283344228.394538, -46373.1796964038, 377826945.047986, -209088.217319699], [-616679.693190829, -369.864520095145, -209088.217319699, 1084.25471015312]]"


Alternatively you can unnest the results for easier reading of output.

In [8]:
%%sql
SELECT unnest(ARRAY['intercept','tax','bath','size']) as attribute,
       unnest(coef) as coefficient,
       unnest(std_err) as standard_error,
       unnest(t_stats) as t_stat,
       unnest(p_values) as pvalue
FROM houses_linregr;

4 rows affected.


attribute,coefficient,standard_error,t_stat,pvalue
intercept,-12849.416896,33453.0344331,-0.384103179688,0.708223134615
tax,28.9613922652,15.8992104964,1.82156166004,0.095800582719
bath,10181.6290713,19437.7710926,0.523806408809,0.610804093527
size,50.5168949154,32.9280231741,1.53416118084,0.153235085548


# 5. Review model with grouping

In [9]:
%%sql
SELECT * FROM houses_linregr_bedroom ORDER BY bedroom;

3 rows affected.


bedroom,coef,r2,std_err,t_stats,p_values,condition_no,num_rows_processed,num_missing_rows_skipped,variance_covariance
2,"[-84242.0345406619, 55.4430144648689, -78966.9753675336, 225.611910021196]",0.968809546465,"[35018.999166635, 19.5731125321026, 23036.8071292953, 49.0448678149636]","[-2.40560942760823, 2.83261103076655, -3.42786111479457, 4.60011251069906]","[0.250804617665626, 0.21605133377637, 0.180704400437667, 0.136272031474349]",10086.104872,5,0,"[[1226330302.63279, -300921.595597853, 551696673.399771, -1544160.63236657], [-300921.595597853, 383.106734194352, -304863.397298569, 323.251642470093], [551696673.399771, -304863.397298569, 530694482.712349, -946345.586402424], [-1544160.63236657, 323.251642470093, -946345.586402424, 2405.39905898726]]"
3,"[-88155.8292501588, 27.1966436294421, 41404.0293363614, 62.6375210753234]",0.841699901311,"[57867.9999702679, 17.8272309154706, 43643.1321511158, 70.8506824864022]","[-1.52339512848988, 1.52556747362489, 0.948695185143874, 0.884077878675973]","[0.188161432894911, 0.187636685729916, 0.38634003237497, 0.417132778705835]",11722.6225642,9,0,"[[3348705420.55893, 433697.545104307, -70253017.4577515, -2593488.13800241], [433697.545104307, 317.810162113512, -90019.0797451145, -529.274668274391], [-70253017.4577515, -90019.0797451147, 1904722983.95976, -2183233.19448568], [-2593488.13800241, -529.27466827439, -2183233.19448568, 5019.81920878898]]"
4,"[0.0112536020318378, 41.4132554771633, 0.0225072040636757, 31.3975496688276]",1.0,"[0.0, 0.0, 0.0, 0.0]","[inf, inf, inf, inf]",None,inf,1,0,"[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]"


# 6. Prediction
Compare predicted price with actual, in the case where not grouping is used.  That is, there is only one regression model for all data.  (This example uses the original data table to perform the prediction. Typically a different test dataset with the same features as the original training dataset would be used for prediction.)

In [10]:
%%sql
SELECT houses.*,
       madlib.linregr_predict( m.coef,
                              ARRAY[1,tax,bath,size]
                             ) as predict,
        price -
          madlib.linregr_predict( m.coef,
                                 ARRAY[1,tax,bath,size] 
                                ) as residual
FROM houses, houses_linregr m ORDER BY id;

15 rows affected.


id,tax,bedroom,bath,price,size,lot,predict,residual
1,590,2,1.0,50000,770,22100,53317.4426966,-3317.44269655
2,1050,3,2.0,85000,1410,12000,109152.124956,-24152.1249556
3,20,3,1.0,22500,1060,3500,51459.3486309,-28959.3486309
4,870,2,2.0,90000,1300,17500,98382.2159072,-8382.21590721
5,1320,3,2.0,133000,1500,30000,121518.22141,11481.7785904
6,1350,2,1.0,90500,820,25700,77853.9455639,12646.0544361
7,2790,3,2.5,260000,2130,25000,201007.926372,58992.0736283
8,680,2,1.0,142500,1170,22000,76130.7259666,66369.2740334
9,1840,3,2.0,160000,1500,19000,136578.145387,23421.8546125
10,3680,4,2.0,240000,2790,20000,255033.901596,-15033.9015962


# 7. Prediction with grouping
Compare predicted price with actual with grouping.  It means a different model is used depending on the number of bedrooms.

In [11]:
%%sql
SELECT houses.*,
       madlib.linregr_predict( m.coef,
                              ARRAY[1,tax,bath,size]
                             ) as predict,
        price -
          madlib.linregr_predict( m.coef,
                                 ARRAY[1,tax,bath,size] 
                                ) as residual
FROM houses, houses_linregr_bedroom m
WHERE houses.bedroom = m.bedroom
ORDER BY id;

15 rows affected.


id,tax,bedroom,bath,price,size,lot,predict,residual
1,590,2,1.0,50000,770,22100,43223.5393424,6776.4606576
2,1050,3,2.0,85000,1410,12000,111527.60995,-26527.6099497
3,20,3,1.0,22500,1060,3500,20187.9052986,2312.09470137
4,870,2,2.0,90000,1300,17500,99354.9203363,-9354.92033626
5,1320,3,2.0,133000,1500,30000,124508.080626,8491.91937359
6,1350,2,1.0,90500,820,25700,96640.8258368,-6140.82583676
7,2790,3,2.5,260000,2130,25000,224650.799707,35349.2002927
8,680,2,1.0,142500,1170,22000,138458.174653,4041.82534729
9,1840,3,2.0,160000,1500,19000,138650.335314,21349.6646863
10,3680,4,2.0,240000,2790,20000,240000.0,0.0
